In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from utils.data.MyDataset import MyDataset
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load pre-trained ResNet18
resnet = models.resnet18(pretrained=True)
# Remove the final classification layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
train_dataset = MyDataset()
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Define model parameters
input_size = 512  # ResNet18 output size
hidden_size = 128  # LSTM hidden size
num_layers = 2  # Number of LSTM layers
num_classes = 4  # Number of output classes

# Initialize LSTM model
lstm_model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    for images, labels in train_loader:
        # Preprocess images
        # Extract features using ResNet18
        features = resnet(images)
        # Prepare LSTM input
        # Reshape features to fit the LSTM input shape
        features = features.view(-1, sequence_length, input_size)
        
        # Forward pass
        outputs = lstm_model(features)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
